In [1]:
import cv2
import dlib
from imutils import face_utils
from scipy.spatial import distance as dist
from pynput.mouse import Button, Controller
import time

# Initializing the mouse controller
mouse = Controller()

# Initializing the dlib's face detector (HOG-based) and facial landmark predictor with the dat's path 
face_detector = dlib.get_frontal_face_detector()
shape_predictor = dlib.shape_predictor("/Users/samikshasharma/Downloads/shape_predictor_68_face_landmarks.dat")

# Constants for the eye aspect ratio (EAR) to detect blinks
EYE_AR_THRESH = 0.25
EYE_AR_CONSEC_FRAMES = 3

# Defining function to compute eye aspect ratio
def eye_aspect_ratio(eye):
    
    # Computing the euclidean distances between the two sets of vertical eye landmarks (y-coordinates)
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    
    # Computing the euclidean distance
    C = dist.euclidean(eye[0], eye[3])
    
    # Computing the eye aspect ratio
    ear = (A + B) / (2.0 * C)
    return ear

# Initializing counters for blinks and display duration
COUNTER = 0
TOTAL = 0
blink_detected_time = 0
DISPLAY_DURATION = 3  # in seconds

# Starting video using webcam
vs = cv2.VideoCapture(0)

while True:
    # Reading the frame from the webcam
    ret, frame = vs.read()
    if not ret:
        break

    # Converting the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the grayscale frame
    rects = face_detector(gray, 0)

    for rect in rects:
        # Determining the facial landmarks for the face region
        face_shape = shape_predictor(gray, rect)
        face_shape = face_utils.shape_to_np(face_shape)

        # Extract the left and right eye coordinates
        leftEye = face_shape[36:42]
        rightEye = face_shape[42:48]

        # Computing the eye aspect ratio for both eyes
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)

        # Average of the eye aspect ratio for both eyes
        ear = (leftEAR + rightEAR) / 2.0

        # Checking if the eye aspect ratio is below the threshold
        if ear < EYE_AR_THRESH:
            COUNTER += 1
        else:
            if COUNTER >= EYE_AR_CONSEC_FRAMES:
                # Performs a click action when a blink is detected
                mouse.click(Button.left, 1)
                TOTAL += 1
                # Set the blink detected time
                blink_detected_time = time.time()
            # Resetting the blink counter
            COUNTER = 0

    # Display "Blink detected" for 3 seconds
    if time.time() - blink_detected_time <= DISPLAY_DURATION:
        cv2.putText(frame, "Blink detected", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

    # Display the frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    # Break the loop if 'q' key is pressed
    if key == ord("q"):
        break

# Releasing the video stream and closing all the windows
vs.release()
cv2.destroyAllWindows()
